In [4]:
from transformers import pipeline
import sqlite3
import spacy
from fuzzywuzzy import process
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the fine-tuned text-to-SQL model
model_name = "cssupport/t5-small-awesome-text-to-sql"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Database connection
DB_PATH = "../logs_simulation/machine_data.db"  # Replace with your SQLite file path

# Synonym mapping for schema
synonym_mapping = {
    "fueltanklevel": "fueltanklevel",
    "fuel level": "fueltanklevel",
    "fuel tank level": "fueltanklevel",
}

schema = {
    "table": "telemetry",  # Replace with your actual table name
    "columns": ["timestamp", "machine_id", "component", "sensor_name", "sensor_value", "sensor_unit"],
}

def find_closest_match(word, choices):
    """Find the closest match using fuzzy matching."""
    match, score = process.extractOne(word, choices)
    return match if score > 80 else word

def preprocess_query(query):
    """Preprocess the natural language query."""
    doc = nlp(query)
    words = [token.text for token in doc]
    # Map synonyms
    mapped_words = [synonym_mapping.get(word.lower(), word) for word in words]
    return " ".join(mapped_words)

def match_schema(query):
    """Match query terms to database schema."""
    matched_columns = [find_closest_match(word, schema["columns"]) for word in query.split()]
    return list(set(matched_columns))

def generate_sql_with_model(query):
    """Generate SQL using a fine-tuned text-to-SQL model."""
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query


def execute_sql(db_path, sql):
    """Execute the SQL query against the SQLite database."""
    if not sql:
        return "Invalid query or unsupported operation."
    
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql)
        results = cursor.fetchall()
        conn.close()
        return results
    except sqlite3.Error as e:
        return f"Database error: {e}"

def natural_language_to_sql_pipeline(query):
    """Complete pipeline from natural language to SQL execution."""
    print(f"Original Query: {query}")

    # Step 1: Preprocess query
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")

    # Step 2: Match schema
    matched_columns = match_schema(processed_query)
    print(f"Matched Columns: {matched_columns}")

    # Step 3: Generate SQL
    sql_query = generate_sql_with_model(processed_query)
    print(f"Generated SQL: {sql_query}")

    # Step 4: Execute SQL
    results = execute_sql(DB_PATH, sql_query)
    print(f"Results: {results}")

    return results

# Example Query
example_query = "What is the average fuel level for the machine?"
results = natural_language_to_sql_pipeline(example_query)


ImportError: 
 requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [1]:
! python -m spacy download en_core_web_sm


/Users/maksimsmirnov/Desktop/MTh/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 379.6 kB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
